In [11]:
# import libraries 
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split

# Set to CPU for now
tf.config.set_visible_devices([], 'GPU')

In [12]:
data = pd.read_csv('/Users/zaravanthoff/Desktop/MasterProject/Datasets/PublicDataset/flickr_logos_27_dataset/flickr_logos_27_dataset_training_set_annotation.txt', sep='\s+',header=None)
# Rename the columns
data.columns = ['Image', 'Class', 'Label', 'Xmin', 'Ymin', 'Xmax', 'Ymax']

folder_path = "/Users/zaravanthoff/Desktop/MasterProject/Datasets/PublicDataset/flickr_logos_27_dataset/flickr_logos_27_dataset_images"

In [13]:
data

,Image,Class,Label,Xmin,Ymin,Xmax,Ymax
0,144503924.jpg,Adidas,1,38,12,234,142
1,2451569770.jpg,Adidas,1,242,208,413,331
2,390321909.jpg,Adidas,1,13,5,89,60
3,4761260517.jpg,Adidas,1,43,122,358,354
4,4763210295.jpg,Adidas,1,83,63,130,93
...,...,...,...,...,...,...,...
4531,2126991906.jpg,Yahoo,6,15,6,253,54
4532,217288720.jpg,Yahoo,6,136,161,304,222
4533,2472817996.jpg,Yahoo,6,2,4,499,106
4534,2514220918.jpg,Yahoo,6,1,69,342,157


In [14]:
# Step 1: Data Preparation (You've already done this to some extent)

# Define the target width and height for resizing
target_width = 416
target_height = 416

resized_images = []
labels = []

# Iterate through each row in the dataset
for index, row in data.iterrows():
    # Extract image filename from the dataset
    image_filename = row[0]  # Accessing by integer index 0
    
    # Construct the full path to the image
    image_path = os.path.join(folder_path, image_filename)
    
    try:
        # Read the image using OpenCV
        image = cv2.imread(image_path)
        
        # Check if the image was read successfully
        if image is None:
            raise FileNotFoundError(f"Unable to read image at path: {image_path}")
        
        # Resize the image to the target size
        resized_image = cv2.resize(image, (target_width, target_height))
        
        # Append resized image to the list
        resized_images.append(resized_image)
        
        # Append label to the list (assuming label is in the second column of the dataframe)
        label = row[2]
        labels.append(label)
        
        print(f"Image processed successfully: {image_filename}")
    except Exception as e:
        print(f"Error processing image at path {image_path}: {str(e)}")


/var/folders/bs/xnv9qfhn03xdhr_rlq0jjbq40000gn/T/ipykernel_89533/428112982.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  image_filename = row[0]  # Accessing by integer index 0
/var/folders/bs/xnv9qfhn03xdhr_rlq0jjbq40000gn/T/ipykernel_89533/428112982.py:33: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = row[2]


Image processed successfully: 144503924.jpg
Image processed successfully: 2451569770.jpg
Image processed successfully: 390321909.jpg
Image processed successfully: 4761260517.jpg
Image processed successfully: 4763210295.jpg
Image processed successfully: 4763210295.jpg
Image processed successfully: 4763210295.jpg
Image processed successfully: 4763210295.jpg
Image processed successfully: 4763210295.jpg
Image processed successfully: 4763210295.jpg
Image processed successfully: 4763210295.jpg
Image processed successfully: 4763210295.jpg
Image processed successfully: 1230939811.jpg
Image processed successfully: 1230939811.jpg
Image processed successfully: 2550056374.jpg
Image processed successfully: 3294282629.jpg
Image processed successfully: 3294282629.jpg
Image processed successfully: 3294282629.jpg
Image processed successfully: 3294282629.jpg
Image processed successfully: 3294282629.jpg
Image processed successfully: 3294282629.jpg
Image processed successfully: 3294282629.jpg
Image proces

In [5]:
# Convert the list of resized images to a NumPy array for easier manipulation
resized_images_array = np.array(resized_images)

# Check the shape of the resized images array
print("Shape of resized images array:", resized_images_array.shape)

Shape of resized images array: (4536, 416, 416, 3)


In [6]:
from sklearn.model_selection import train_test_split

# Define the percentage of data for training, validation, and testing
train_percent = 0.8  # 80% of data for training
val_percent = 0.1    # 10% of data for validation
test_percent = 0.1   # 10% of data for testing

# Split the data into training and the rest
train_data, temp_data = train_test_split(data, train_size=train_percent, random_state=42)

# Split the rest into validation and testing
val_data, test_data = train_test_split(temp_data, train_size=val_percent/(val_percent + test_percent), random_state=42)

# Display the number of samples in each set
print("Number of samples in training set:", len(train_data))
print("Number of samples in validation set:", len(val_data))
print("Number of samples in test set:", len(test_data))

# Display the number of unique images in each set
print("Number of unique images in training set:", len(train_data['Image'].unique()))
print("Number of unique images in validation set:", len(val_data['Image'].unique()))
print("Number of unique images in test set:", len(test_data['Image'].unique()))

Number of samples in training set: 3628
Number of samples in validation set: 454
Number of samples in test set: 454
Number of unique images in training set: 776
Number of unique images in validation set: 284
Number of unique images in test set: 284


In [7]:
# Check the data types of the columns in train_data
print("Data types in train_data:")
print(train_data.dtypes)

# Check the data types of the columns in val_data
print("\nData types in val_data:")
print(val_data.dtypes)

Data types in train_data:
Image    object
Class    object
Label     int64
Xmin      int64
Ymin      int64
Xmax      int64
Ymax      int64
dtype: object

Data types in val_data:
Image    object
Class    object
Label     int64
Xmin      int64
Ymin      int64
Xmax      int64
Ymax      int64
dtype: object


In [8]:
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Load and preprocess image data
train_images = []
val_images = []

for img_path in train_data['Image']:
    img = cv2.imread(os.path.join(folder_path, img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, (target_width, target_height))  # Resize image
    img = img.astype(np.float32) / 255.0  # Normalize pixel values
    train_images.append(img)

for img_path in val_data['Image']:
    img = cv2.imread(os.path.join(folder_path, img_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, (target_width, target_height))  # Resize image
    img = img.astype(np.float32) / 255.0  # Normalize pixel values
    val_images.append(img)

# Convert lists to numpy arrays
train_images = np.array(train_images)
val_images = np.array(val_images)

# Preprocess class data
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_data['Class'])
val_labels = label_encoder.transform(val_data['Class'])
num_classes = len(label_encoder.classes_)
train_labels_one_hot = to_categorical(train_labels, num_classes=num_classes).astype(np.float32)  # Ensure float32
val_labels_one_hot = to_categorical(val_labels, num_classes=num_classes).astype(np.float32)  # Ensure float32

---------------

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load pre-trained model (excluding top layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(target_height, target_width, 3))

# Optionally freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Define the model architecture
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Define ModelCheckpoint callback to save the best model during training
checkpoint_path = "model_checkpoint.keras"
model_checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Train the model with early stopping and model checkpointing
history = model.fit(train_images, train_labels_one_hot, epochs=10, batch_size=32, 
                    validation_data=(val_images, val_labels_one_hot),
                    callbacks=[early_stopping, model_checkpoint])

Epoch 1/10
  4/114 ━━━━━━━━━━━━━━━━━━━━ 25:25 14s/step - accuracy: 0.0710 - loss: 5.5657

KeyboardInterrupt: 

In [ ]:
# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(target_height, target_width, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels_one_hot, epochs=10, batch_size=32, validation_data=(val_images, val_labels_one_hot))

--------------

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

# Load pre-trained model (excluding top layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Optionally freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Add new classification layers
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)

# Combine base model with new classification layers
model = models.Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history = model.fit(train_images, train_labels_one_hot, epochs=10, batch_size=32, validation_data=(val_images, val_labels_one_hot))


TypeError: VGG16() got an unexpected keyword argument 'output_shape'

--------------------

In [ ]:
def yolo_model(input_shape, num_classes):
    # Input layer
    inputs = tf.keras.Input(shape=input_shape)

    # YOLO backbone
    backbone = tf.keras.applications.MobileNetV2(input_tensor=inputs, include_top=False, weights='imagenet')
    
    # Freeze the backbone layers
    backbone.trainable = False

    # YOLO head
    x = layers.GlobalAveragePooling2D()(backbone.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    
    # Output layer for bounding box coordinates
    bbox_output = layers.Dense(4, activation='linear', name='bbox_output')(x)  # Linear activation for regression
    
    # Output layer for class prediction
    class_output = layers.Dense(num_classes, activation='softmax', name='class_output')(x)
    
    # Create the model
    model = tf.keras.Model(inputs=inputs, outputs=[bbox_output, class_output])

    return model

def bounding_box_loss(y_true, y_pred):
    # Compute the Mean Squared Error (MSE) loss for bounding box regression
    mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
    loss = tf.reduce_mean(mse_loss)
    return loss

# Define input shape and number of classes
input_shape = (416, 416, 3)  # Example input shape (adjust as needed)
num_classes = 27  # Example number of classes (adjust as needed)

# Create the YOLO model
model = yolo_model(input_shape, num_classes)

# Compile the model with custom loss function
model.compile(optimizer='adam', loss={'bbox_output': bounding_box_loss, 'class_output': 'categorical_crossentropy'}, metrics={'class_output': 'accuracy'})

# Display the model summary
# model.summary()
